In [1]:
import requests
import pandas as pd
from pathlib import Path
import logging
import os
import sys
from scipy.stats import zscore
import dataframe_image as dfi

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
from config.config import Config
from visualization.visualization import make_highlighted_table, team_styled_table

config = Config()

In [2]:
def fetch_all_blcs_season_2(group_id, token=config._ballchasing_token):

    url = f"https://ballchasing.com/api/groups/{group_id}"
    headers = {"Authorization": token}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    df = pd.json_normalize(data, record_path=["players"])
    return df

In [3]:
df = fetch_all_blcs_season_2(config.all_blcs_season_2_matches_link)

In [4]:
df.sort_values(by="team")

,platform,id,name,team,cumulative.games,cumulative.wins,cumulative.win_percentage,cumulative.play_duration,cumulative.core.shots,cumulative.core.shots_against,...,game_average.positioning.time_farthest_from_ball,game_average.positioning.percent_defensive_third,game_average.positioning.percent_offensive_third,game_average.positioning.percent_neutral_third,game_average.positioning.percent_defensive_half,game_average.positioning.percent_offensive_half,game_average.positioning.percent_behind_ball,game_average.positioning.percent_infront_ball,game_average.demo.inflicted,game_average.demo.taken
21,Steam,76561198395894083,nyk,BECKYARDIGANS,35,13,37.142857,12825,120,284,...,110.397140,47.768616,21.863562,30.367819,63.382435,36.617565,72.399600,27.600399,1.028571,0.800000
15,Epic,0c0d9a0287d14575b62c5c314219f8f0,beckham1717,BECKYARDIGANS,35,13,37.142857,12825,76,284,...,127.705720,42.673412,23.461860,33.864730,59.260780,40.739220,68.709280,31.290722,0.600000,0.942857
9,Steam,76561198262801476,Supe,BECKYARDIGANS,35,13,37.142857,12825,95,284,...,118.285710,46.467484,22.077406,31.455112,62.842403,37.157597,69.504410,30.495586,0.771429,1.257143
3,Steam,76561198949468845,Duri,DIDDLERS,54,31,57.407406,20000,165,373,...,109.566690,45.789170,22.874939,31.335888,62.648735,37.351265,71.470000,28.529993,1.074074,1.111111
22,Steam,76561198151665626,ro,DIDDLERS,54,31,57.407406,20000,111,373,...,132.885180,46.900475,22.212650,30.886875,63.398327,36.601680,69.557655,30.442335,0.759259,0.962963
13,Steam,76561198298956453,Xero,DIDDLERS,54,31,57.407406,20000,156,373,...,121.318535,44.331090,22.872120,32.796790,61.436430,38.563564,68.394900,31.605108,1.259259,1.111111
2,Steam,76561198976276890,Distill?,EXECUTIVE PROJEC,44,21,47.727272,15874,68,359,...,119.472725,47.096570,19.195803,33.707630,64.669830,35.330166,72.811554,27.188442,0.431818,0.727273
18,Epic,2c93f906466240ee97bfdaeda4c5d9c7,jeridinho.,EXECUTIVE PROJEC,44,21,47.727272,15874,184,359,...,109.497770,49.675140,20.942436,29.382418,65.621826,34.378174,77.049225,22.950771,0.795455,1.022727
8,Steam,76561198389804286,Ray,EXECUTIVE PROJEC,44,21,47.727272,15874,102,359,...,122.152275,47.595860,20.195183,32.208954,64.932220,35.067780,71.296080,28.703928,1.204546,1.068182
5,Epic,17ba1b410d0e4131b4ea04fc63c777f6,Kyzeke,KILLER BS,37,16,43.243244,13630,98,267,...,101.159460,45.576817,22.311186,32.111996,62.268074,37.731930,70.952060,29.047941,1.486486,0.972973


In [5]:
def process():
    logging.info("Processing player data")
    df = fetch_all_blcs_season_2(config.all_blcs_season_2_matches_link)
    
    features_to_keep = [
        "name",
        "team",
        "game_average.core.score",
        "game_average.core.goals",
        "game_average.core.assists",
        "game_average.core.saves",
        "game_average.core.shots",
        "game_average.core.shooting_percentage",
        "game_average.demo.inflicted",
        "game_average.demo.taken",
        "game_average.boost.amount_stolen_big",
        "game_average.boost.amount_stolen_small",
    ]

    df_final = df[features_to_keep].copy()
    df_final.columns = ["Player", "Team", "Avg Score", "Goals Per Game", "Assists Per Game", "Saves Per Game", "Shots Per Game", "Shooting %", "Demos Inf. Per Game", "Demos Taken Per Game", "Big Boost Stolen", "Small Boost Stolen"]

    df_final = df_final.drop("Team", axis=1)
    logging.info("Calculating Z-scores and Dominance Quotient for players...")
    df_final["Avg Score Zscore"] = round(zscore(df_final["Avg Score"]) * config.avg_score, 2)
    df_final["Goals Per Game Zscore"] = round(zscore(df_final["Goals Per Game"]) * config.goals_per_game, 2)
    df_final["Assists Per Game Zscore"] = round(zscore(df_final["Assists Per Game"]) * config.assists_per_game, 2)
    df_final["Saves Per Game Zscore"] = round(zscore(df_final["Saves Per Game"]) * config.saves_per_game, 2)
    df_final["Shots Per Game Zscore"] = round(zscore(df_final["Shots Per Game"]) * config.shots_per_game, 2)
    df_final["Demos Inf. Per Game Zscore"] = round(zscore(df_final["Demos Inf. Per Game"]) * config.demos_per_games, 2)
    df_final["Demos Taken Per Game Zscore"] = round(zscore(df_final["Demos Taken Per Game"]) * config.demos_taken_per_game, 2)
    df_final["Big Boost Stolen Zscore"] = round(zscore(df_final["Big Boost Stolen"]) * config.count_big_pads_stolen_per_game, 2)
    df_final["Small Boost Stolen Zscore"] = round(zscore(df_final["Small Boost Stolen"]) * config.count_small_pads_stolen_per_game, 2)
    df_final["Shooting %"] = df_final["Shooting %"] / 100

    # Calculate Dominance Quotient
    dq_summation = [i for i in df_final.columns.tolist() if "Zscore" in i]
    df_final["Dominance Quotient"] = (df_final[dq_summation].sum(axis=1) + 2) * config.dominance_quotient_multiplier

    df_final = df_final[["Player", "Dominance Quotient", "Avg Score", "Goals Per Game", "Assists Per Game", "Saves Per Game", "Shots Per Game", "Shooting %", "Demos Inf. Per Game", "Demos Taken Per Game", "Big Boost Stolen", "Small Boost Stolen"]]

    df_final = df_final.sort_values(by="Dominance Quotient", ascending=False).reset_index(drop=True)
    df_final.index += 1

    return df_final

In [6]:
df = process()
df.to_parquet("../data/parquet/season_2_overall.parquet")

In [7]:
df

,Player,Dominance Quotient,Avg Score,Goals Per Game,Assists Per Game,Saves Per Game,Shots Per Game,Shooting %,Demos Inf. Per Game,Demos Taken Per Game,Big Boost Stolen,Small Boost Stolen
1,jeridinho.,77.4,581.31820,1.409091,0.409091,2.840909,4.181818,0.336957,0.795455,1.022727,390.31818,191.02272
2,desi,65.8,538.88370,1.186047,0.418605,2.255814,3.720930,0.318750,1.441860,0.813953,313.74417,248.00000
3,Bax,64.0,452.82500,0.775000,0.900000,2.050000,3.450000,0.224638,0.975000,1.025000,323.90000,235.72500
4,nyk,59.2,456.74286,1.028571,0.571429,1.942857,3.428571,0.300000,1.028571,0.800000,318.28570,258.62857
5,Paperclip94,55.2,448.84314,1.019608,0.529412,1.745098,3.274510,0.311377,0.784314,1.058824,252.70589,233.90196
6,Duri,50.6,391.61110,0.888889,0.574074,1.407407,3.055556,0.290909,1.074074,1.111111,310.27777,279.70370
7,Xero,47.4,351.57407,0.870370,0.611111,1.055556,2.888889,0.301282,1.259259,1.111111,341.35184,247.46297
8,Weathers,45.0,384.33334,0.848485,0.363636,1.636364,2.636364,0.321839,1.848485,1.060606,258.30304,282.63635
9,buckystyle,44.6,366.67570,0.864865,0.324324,1.405405,3.351351,0.258065,1.216216,1.243243,409.51352,245.62163
10,Pullis.,41.8,346.90910,0.696970,0.606061,1.636364,2.121212,0.328571,0.636364,1.303030,222.36363,215.42424


In [8]:
styled_df = make_highlighted_table(df)
dfi.export(styled_df, "../images/season_2_overall.png")

In [9]:
styled_df

,Player,Dominance Quotient,Avg Score,Goals Per Game,Assists Per Game,Saves Per Game,Shots Per Game,Shooting %,Demos Inf. Per Game,Demos Taken Per Game,Big Boost Stolen,Small Boost Stolen
1,jeridinho.,77.40,581.32,1.41,0.41,2.84,4.18,33.70%,0.80,1.02,390.32,191.02
2,desi,65.80,538.88,1.19,0.42,2.26,3.72,31.87%,1.44,0.81,313.74,248.00
3,Bax,64.00,452.82,0.78,0.90,2.05,3.45,22.46%,0.97,1.02,323.90,235.72
4,nyk,59.20,456.74,1.03,0.57,1.94,3.43,30.00%,1.03,0.80,318.29,258.63
5,Paperclip94,55.20,448.84,1.02,0.53,1.75,3.27,31.14%,0.78,1.06,252.71,233.90
6,Duri,50.60,391.61,0.89,0.57,1.41,3.06,29.09%,1.07,1.11,310.28,279.70
7,Xero,47.40,351.57,0.87,0.61,1.06,2.89,30.13%,1.26,1.11,341.35,247.46
8,Weathers,45.00,384.33,0.85,0.36,1.64,2.64,32.18%,1.85,1.06,258.30,282.64
9,buckystyle,44.60,366.68,0.86,0.32,1.41,3.35,25.81%,1.22,1.24,409.51,245.62
10,Pullis.,41.80,346.91,0.70,0.61,1.64,2.12,32.86%,0.64,1.30,222.36,215.42
